# Precipitation climate index
Compute the annual count of days when precipitation exceeds a given threshold (e.g., 10mm)

Let RR<sub>ij</sub> be the daily precipitation amount on day i in period j. Count the number of days where 

\begin{equation*} 
RRij ≥ 10mm
\end{equation*}

The notebook will exploit:
- an **intake** catalog to search and discover CMIP6 data
- **xarray** to compute the climate index
- **Cartopy** and **Matplotlib** to plot the results.

Make sure to select **Python 3** as Notebook Kernel.

Recommended profile: LARGE

Import the main Python modules

In [ ]:
import os, intake
import multiprocessing
import dask
import xarray as xr
import numpy as np
import pandas as pd
import ipywidgets as widgets
from os.path import expanduser
home = expanduser("~")

**Search and discover datasets by using *intake-esm***

We use the **intake-esm** (https://github.com/intake/intake-esm) data cataloging utility to search, discover, access and load data.

An ESM (Earth System Model) collection definition file is used by intake-esm to establish a link to a database (CSV file) that contains data assets locations and associated metadata (i.e., which experiment, model, ...).

The ESM collection file (**CMIP6_ESM_colletion_file.json**) is located under the **data** folder.

In [ ]:
esm_file = home+"/data/CMIP6_ESM_colletion_file.json"
col = intake.open_esm_datastore(esm_file)
col

The output shows how the CMIP6 catalog is organized. Specifically, it provides some aggregated information over the metadata fields, which represents the core components to describe CMIP6 datasets according to the data reference syntax.

The in-memory representation for the catalog is a Pandas DataFrame. We can inspect it with **col.df**.

In [ ]:
col.df.head()

Get unique values for **models** and **experiments** in the catalog.

In [ ]:
uniques = col.unique().loc[["source_id","experiment_id"]].tolist()
import pprint
pprint.pprint(uniques, compact=True)

Select one of the models and experiments available

In [ ]:
sources = widgets.Dropdown(
    options=uniques[0],
    value="CMCC-CM2-SR5",
    description='Source:',
    disabled=False,
)
display(sources)

experiments = widgets.Dropdown(
    options=uniques[1],
    value="ssp245",
    description='Experiment:',
    disabled=False,
)
display(experiments)

In [ ]:
print(sources.value)
print(experiments.value)

Execute a query against the catalog to retrieve entries satisfyng the search criteria

In [ ]:
query = dict( experiment_id=experiments.value,
             source_id=sources.value, 
             variable_id="pr",
             table_id="day"
)
cat = col.search(**query)
cat.df

Load data assets into xarray datasets

In [ ]:
xrdsetdict = cat.to_dataset_dict()
xrdsetdict

In [ ]:
dspr = xrdsetdict[[key for key in xrdsetdict.keys()] [0]]
dspr

Select the time ranges of interest among those available for the filtered data and run the next cell to check the selection.

In [ ]:
start_time = widgets.Dropdown(
    options=np.unique(dspr.time.dt.year.values),
    value=[np.unique(dspr.time.dt.year.values)[0]],
    description='Start Time:',
    disabled=False,
)
display(start_time)

end_time = widgets.Dropdown(
    options=np.unique(dspr.time.dt.year.values),
    value=2050,
    description='End Time:',
    disabled=False,
)
display(end_time)

In [ ]:
print(start_time.value)
print(end_time.value)
tashist = dspr.isel(member_id=0).sel(time=dspr.time.dt.year.isin(range(start_time.value, end_time.value+1)))
tashist

Count the number of days (on yearly basis) exceeding the given threshold

**NOTE:** The precipitation is expressed in *kg m-2 s-1*, so a scale factor is needed to convert the unit in *mm/day*.

In [ ]:
# EUROPE
north= 70.94985882321207
south= 35.049860308608224
east= 40
west= -24.950139509199722

def annual_rnmm(tashist, mm_threshold):
    
    def _count_rnmm(x, axis):
            factor=86400  # 1 kg/m2/s = 86400 mm/day
            return np.sum(x >= (mm_threshold/factor), axis=axis)
        
    total = tashist['pr'].sel(lat=slice(south,north), lon=slice(west,east)).groupby('time.year').reduce(_count_rnmm)
    return total

mm_threshold=10.0
rnmm = annual_rnmm(tashist, mm_threshold)
result = rnmm.compute()
rnmm

Get coordinate arrays

In [ ]:
rnmm.coords

Select one year (whole spatial domain)

In [ ]:
year=2050
rnmm_one_year=rnmm.sel(year=year)
rnmm_one_year

Plot the results on a map

In [ ]:
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.util import add_cyclic_point
import numpy as np
import warnings
warnings.filterwarnings("ignore")

lat = rnmm_one_year['lat'].values
lon = rnmm_one_year['lon'].values
var = rnmm_one_year.values

fig = plt.figure(figsize=(15, 6), dpi=100)

#Add Geo axes to the figure with the specified projection (PlateCarree)
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)

#Draw coastline and gridlines
ax.coastlines()
gl = ax.gridlines(crs=projection, draw_labels=True, linewidth=1, color='black', alpha=0.9, linestyle=':')
gl.xlabels_top = False
gl.ylabels_right = False

var = np.reshape(var, (len(lat), len(lon)))

#Wraparound points in longitude
var_cyclic, lon_cyclic = add_cyclic_point(var, coord=lon)
x, y = np.meshgrid(lon_cyclic,lat)

#Define color levels for color bar
levStep = (np.max(var)-np.min(var))/15
clevs = np.arange(np.min(var),np.max(var)+levStep,levStep)

#Set filled contour plot
cnplot = ax.contourf(x, y, var_cyclic, clevs, transform=projection,cmap=plt.cm.jet)
plt.colorbar(cnplot,ax=ax)

#ax.set_aspect('auto', adjustable=None)
plt.title('R10mm (year '+str(year)+')')
plt.show()

Select one spatial point (whole time series)

In [ ]:
rnmm_one_point=rnmm.sel(lat=54, lon=7,method='nearest')
rnmm_one_point

Plot results on a graph

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
y = rnmm_one_point.values
x = rnmm_one_point.year.values
plt.figure(figsize=(11, 5), dpi=100)
plt.plot(x, y)

plt.ylabel("mm")
plt.xlabel("years")
plt.title("(lat,lon) = ("+str(rnmm_one_point.lat.values)+',' +str(rnmm_one_point.lon.values)+")")
plt.xticks([p for p in x[::1]], x[::1], rotation='vertical')
plt.show()